In [ ]:
import pandas as pd
import numpy as np

# Specify the path to the pickle file
data_file_path = '../Dataset/Dementia_paper_dataset_data.pkl'
info_file_path = '../Dataset/Dementia_paper_dataset_info.pkl'

# Open the pickle file in read mode
with open(data_file_path, 'rb') as file:
    # Load the data from the pickle file
    data = pd.read_pickle(file)

# Open the pickle file in read mode
with open(info_file_path, 'rb') as file:
    # Load the data from the pickle file
    info = pd.read_pickle(file)

# Display the first few rows of the data
data['train'].head()

In [76]:
# Import the pandas library
import pandas as pd

# Specify the file path for the Excel file
excel_file_path = '../Dataset/Dementia_paper_dataset_data.xlsx'

# Create an ExcelWriter object
writer = pd.ExcelWriter(excel_file_path, engine='openpyxl')

# Loop through the dictionary items and save each dataframe as a separate sheet in the Excel file
for key, value in info.items():
    if key != 'Info':
        # Write the dataframe to the Excel file
        value.to_excel(writer, sheet_name=key, index=False)

# Save the Excel file
writer.close()



HC: 0, MCI: 1, Dementia: 2

In [ ]:
import numpy as np
# Dataframe to numpy array in each dictionary
data_ndarrays = {}
for key, value in data.items():
    # Detect if the key is 'Info' and skip it
    if key != 'Info':
        # Drop the 'ID' and 'Task' columns if they exist
        if 'ID' in data[key].columns:
            data[key].drop('ID', axis=1, inplace=True)
        if 'Task' in data[key].columns:
            data[key].drop('Task', axis=1, inplace=True)  
        data_ndarrays[key] = value.values
    else:
        data_ndarrays[key] = value

label_ndarrays = {}
label_ndarrays_CInonCI = {}
for key, value in info.items():
    if key != 'Info':
        label_ndarrays[key] = value['Label'].values
        label_ndarrays_CInonCI[key] = value['Label'].values
    else:
        label_ndarrays[key] = value
        label_ndarrays_CInonCI[key] = value

# Replace 2 with 1 in label_ndarrays
label_ndarrays_CInonCI = {key: np.where(value == 2, 1, value) for key, value in label_ndarrays_CInonCI.items()}

print(label_ndarrays['train'])
label_ndarrays_CInonCI['train']

In [ ]:
# classifying the data using LDA and LOO-CV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm import tqdm, trange
import os
from feature_related import feature_selection as f_selection  # Import the feature_selection module

lda = LinearDiscriminantAnalysis()
qda = QuadraticDiscriminantAnalysis()
# Create the pipeline
pipeline = Pipeline([
    ('lda', lda)
])

looCV = LeaveOneOut()

# Train & Valid data organizing
data_train_valid = np.concatenate([data_ndarrays['train'], data_ndarrays['valid']], axis=0)
label_train_valid = np.concatenate([label_ndarrays_CInonCI['train'], label_ndarrays_CInonCI['valid']], axis=0)

# Feature selection - filter_based
f_selection = f_selection()
fisher_scores = f_selection.fisher_score(data_train_valid, label_train_valid)
fisher_idx = np.argsort(fisher_scores)[::-1] # sort in descending order

# Loop through the indices the looCV.split() method returns
df_accCV =list()
max_dim = int(label_train_valid.shape[0]-1)  # Define the maximum number of features
for idx_AOFI in tqdm(range(1, max_dim + 1)):
    data_CV = data_train_valid[:, fisher_idx[:idx_AOFI]]
    # print(data_CV.shape)
    accCV = 0
    predicted_labels = []
    for i, (train_idx, valid_idx) in enumerate(looCV.split(data_train_valid)):
        # Train the model
        pipeline.fit(data_CV[train_idx], label_train_valid[train_idx])
        # Test the model
        predicted_labels.append(pipeline.predict(data_CV[valid_idx]))
        # print('Prediction:', score, end=' ')
        # print('True label:', label_ndarrays_CInonCI['train'][valid_idx])

    predicted_labels = np.concatenate(predicted_labels)

    # Calculate true positive rate (TPR)
    TP = sum((label_train_valid == 1) & (predicted_labels == 1))
    FN = sum((label_train_valid == 1) & (predicted_labels == 0))
    TPR = TP / (TP + FN)

    # Calculate false positive rate (FPR)
    FP = sum((label_train_valid == 0) & (predicted_labels == 1))
    TN = sum((label_train_valid == 0) & (predicted_labels == 0))
    TNR = TN / (FP + TN)

    balanced_accuracy = (TPR + TNR) / 2
    # Append the loop index and accCV to the dataframe
    df_accCV.append({'#features': idx_AOFI, 'accCV(balanced)': balanced_accuracy})


# Specify the folder path
folder_path = '../Results/Classification/LDA/AOFI/'

# Create the folder if it does not exist
if not os.path.exists(folder_path):    os.makedirs(folder_path)

# Save the dataframe to csv
df_accCV = pd.DataFrame(df_accCV) # Convert the list to a dataframe
df_accCV.to_csv(os.path.join(folder_path, 'df_accCV.csv'), index=False)

In [70]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd

# Create the SVM classifier
svm = SVC()


# Define the parameter grid for GridSearchCV
gamma_range = np.linspace(-100,100,41) #-100,-95,...,95,100
gamma_range = 1.05**gamma_range # 1.05^-100,1.05^-95,...,1.05^95,1.05^100
gamma_range = 1 / 2*(np.square(gamma_range))  # gamma = 1 / (2*sigma)^2, based on the SVC documentation
gamma_range = gamma_range.tolist()
C_range = [1, 10, 100, 500, 1000]
param_grid = {'C': C_range, 'gamma': gamma_range, 'kernel': ['rbf']}

# Loop through the indices the looCV.split() method returns
df_accCV =list()
max_dim = int(label_train_valid.shape[0]/2)  # Define the maximum number of features
for idx_AOFI in tqdm(range(1, max_dim + 1)):
    data_CV = data_train_valid[:, fisher_idx[:idx_AOFI]]
    
    # Create the GridSearchCV object
    grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=10)

    # Train the model with gridsearchCV
    grid_search.fit(data_CV, label_train_valid)

    # Get the best parameters and best score
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    df_accCV.append({'#features': idx_AOFI, 'best_score': best_score, 'best_params': best_params})

# Specify the folder path
folder_path = '../Results/Classification/SVM/AOFI_smallerGrid_10Fold/'

# Create the folder if it does not exist
if not os.path.exists(folder_path):    os.makedirs(folder_path)

# Save the dataframe to csv
df_accCV = pd.DataFrame(df_accCV) # Convert the list to a dataframe
df_accCV.to_csv(os.path.join(folder_path, 'df_accCV.csv'), index=False)

# Create a dictionary with 'fisher_idx' and 'feature_type' as keys
fisher_idx_DF = {'fisher_idx': fisher_idx, 'feature_type': data['train'].columns[fisher_idx]}

# Convert the dictionary to a DataFrame
fisher_idx_DF = pd.DataFrame(fisher_idx_DF)

# Save the DataFrame to a .csv file
fisher_idx_DF.to_csv(os.path.join(folder_path, 'fisher_idx_series.csv'), index=False)

100%|██████████| 105/105 [07:59<00:00,  4.57s/it]
